In [6]:
import os
import numpy as np
import rasterio
import matplotlib.image

from pathlib import Path


In [7]:
files = {}

for f in os.listdir("data"):
    if ".jp2" in f:
        parts = f.split("_")
        if parts[0] not in files:
            files[parts[0]] = {}
        if parts[1] not in files[parts[0]]:
            files[parts[0]][parts[1]] = {}
        files[parts[0]][parts[1]][parts[2]] = f"data/{f}"


In [8]:
dates = []
for f in files.values():
    dates += list(f.keys())
dates = sorted(list(set(dates)))
    

In [9]:
dates


['20150711T100006',
 '20150804T094006',
 '20150807T095006',
 '20150808T092006',
 '20150811T093006',
 '20150817T095016',
 '20150820T100016',
 '20150821T093006',
 '20150824T094006',
 '20150910T092726',
 '20150913T094016',
 '20150919T100016',
 '20150930T092946',
 '20151003T094016',
 '20151006T095016',
 '20151208T100412',
 '20151228T100422',
 '20160101T094412',
 '20160107T100412',
 '20160108T093352',
 '20160216T100102',
 '20160324T095012',
 '20160328T093002',
 '20160404T091712',
 '20160407T093032',
 '20160427T093042',
 '20160430T094032',
 '20160506T100032',
 '20160507T093042',
 '20160510T094042',
 '20160520T094042',
 '20160524T092032',
 '20160527T093042',
 '20160530T094042',
 '20160625T100032',
 '20160725T100032',
 '20160824T100032',
 '20160828T094032',
 '20160910T095032',
 '20160911T092032',
 '20160913T100022',
 '20160914T093032',
 '20160927T094032',
 '20161010T095022',
 '20161013T100022',
 '20161017T094032',
 '20161129T095342',
 '20161212T100412',
 '20170105T094402',
 '20170111T100351',


In [10]:
offsets = {
    'T34UDG': (30000+200, 0),
    'T34UEG': (30000+200, 10000),
    'T34UFG': (30000+200, 20000),
    'T34VDH': (20000+200, 0),
    'T34VDJ': (10000+200, 0),
    'T34VDK': (0+200, 0), # top left
    'T34VEH': (20000+200, 10000),
    'T34VEJ': (10000+200, 10000),
    'T34VEK': (0+200, 10000),
    'T34VFH': (20000+200, 20000),
    'T34VFJ': (10000+200, 20000),
    'T34VFK': (0+200, 20000),
    'T35ULB': (30000, 30000-4550),
    'T35UMB': (30000, 40000-4550),
    'T35UNB': (30000, 50000-4550),
    'T35VLC': (20000, 30000-4550),
    'T35VLD': (10000, 30000-4550),
    'T35VLE': (0, 30000-4550),
    'T35VMC': (20000, 40000-4550),
    'T35VMD': (10000, 40000-4550),
    'T35VME': (0, 40000-4550),
    'T35VNC': (20000, 50000-4550),
    'T35VND': (10000, 50000-4550),
    'T35VNE': (0, 50000-4550)
}


In [11]:
rgb_composite = np.dstack(
    (np.full((40980, 57410), 0.0), np.full((40980, 57410), 0.0), np.full((40980, 57410), 0.0))
)


In [ ]:
gain = 2

for d in dates:
    print(d)
    for ki, vi in files.items():
        if d in vi:
            print("matched", ki, d)
            fj = vi[d]
            try:
                try:
                    #print("reading SCL", fj["SCL"])
                    scl = np.repeat(np.repeat(rasterio.open(fj["SCL"], driver="JP2OpenJPEG").read(1), 2, 0), 2, 1)
                except BaseException as e:
                    if os.path.isfile(fj["SCL"]):
                        print("REMOVED", fj["SCL"])
                        os.remove(fj["SCL"])
                    raise e
                
                try:
                    #print("reading red", fj["B04"])
                    c_red = np.clip(rasterio.open(fj["B04"], driver="JP2OpenJPEG").read(1)*gain/10000, 0, 1)
                except BaseException as e:
                    if os.path.isfile(fj["B04"]):
                        print("REMOVED", fj["B04"])
                        os.remove(fj["B04"])
                    raise e
                try:
                    #print("reading green", fj["B03"])
                    c_green = np.clip(rasterio.open(fj["B03"], driver="JP2OpenJPEG").read(1)*gain/10000, 0, 1)             
                except BaseException as e:
                    if os.path.isfile(fj["B03"]):
                        print("REMOVED", fj["B03"])
                        os.remove(fj["B03"])
                    raise e
                try:
                    #print("reading blue", fj["B02"])
                    c_blue = np.clip(rasterio.open(fj["B02"], driver="JP2OpenJPEG").read(1)*gain/10000, 0, 1)              
                except BaseException as e:
                    if os.path.isfile(fj["B02"]):
                        print("REMOVED", fj["B02"])
                        os.remove(fj["B02"])
                    raise e

                mask = ((scl != 0) & (scl != 1))
                #print(mask)
                #red[mask] = c_red[mask]
                #green[mask] = c_green[mask]
                #blue[mask] = c_blue[mask]
                #print("applying red")
                rgb_composite[offsets[ki][0]:offsets[ki][0]+mask.shape[0], offsets[ki][1]:offsets[ki][1]+mask.shape[1], 0][mask] = c_red[mask]
                #print("applying green")
                rgb_composite[offsets[ki][0]:offsets[ki][0]+mask.shape[0], offsets[ki][1]:offsets[ki][1]+mask.shape[1], 1][mask] = c_green[mask]
                #print("applying blue")
                rgb_composite[offsets[ki][0]:offsets[ki][0]+mask.shape[0], offsets[ki][1]:offsets[ki][1]+mask.shape[1], 2][mask] = c_green[mask]

            except BaseException as e:
                print(f"ERROR: {ki} {d} failed with {e}")
            break
        else:
            print("no match", ki, d)
    #print("saving composite")
    matplotlib.image.imsave(f"{Path.home()}/Projs/bulbulis/notebooks/true_color_frames/{d}.jpeg", rgb_composite)


20150711T100006
no match T34VFH 20150711T100006
no match T34UDG 20150711T100006
no match T35UMB 20150711T100006
no match T34VDH 20150711T100006
no match T34VDJ 20150711T100006
no match T34VDK 20150711T100006
no match T34VEH 20150711T100006
no match T35VND 20150711T100006
no match T34UEG 20150711T100006
no match T35ULB 20150711T100006
matched T34VFJ 20150711T100006
reading SCL data/T34VFJ_20150711T100006_SCL_20m.jp2
reading red data/T34VFJ_20150711T100006_B04_10m.jp2
reading green data/T34VFJ_20150711T100006_B03_10m.jp2
reading blue data/T34VFJ_20150711T100006_B02_10m.jp2
applying red
applying green
applying blue
saving composite
20150804T094006
matched T34VFH 20150804T094006
reading SCL data/T34VFH_20150804T094006_SCL_20m.jp2
reading red data/T34VFH_20150804T094006_B04_10m.jp2
reading green data/T34VFH_20150804T094006_B03_10m.jp2
reading blue data/T34VFH_20150804T094006_B02_10m.jp2
applying red
applying green
applying blue
saving composite
20150807T095006
no match T34VFH 20150807T09500